# Clustering Crypto

In [180]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [181]:
# Use the following endpoint to fetch json data
import json
url = "https://min-api.cryptocompare.com/data/all/coinlist"
req = requests.get(url)


In [182]:
# Create a DataFrame 
txt = req.json()
url_df = pd.DataFrame(txt)
url_df.head()

,Response,Message,Data,RateLimit,HasWarning,Type
42,Success,Coin list succesfully returned!,"{'Id': '4321', 'Url': '/coins/42/overview', 'I...",NaN,False,100
300,Success,Coin list succesfully returned!,"{'Id': '749869', 'Url': '/coins/300/overview',...",NaN,False,100
365,Success,Coin list succesfully returned!,"{'Id': '33639', 'Url': '/coins/365/overview', ...",NaN,False,100
404,Success,Coin list succesfully returned!,"{'Id': '21227', 'Url': '/coins/404/overview', ...",NaN,False,100
433,Success,Coin list succesfully returned!,"{'Id': '926547', 'Url': '/coins/433/overview',...",NaN,False,100


In [183]:
# Alternatively, use the provided csv file:

# Create a DataFrame
csv_df = pd.read_csv(".\\Resources\crypto_data.csv", index_col=0)
csv_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [184]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
csv_df = csv_df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply']]
csv_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000


In [185]:
# Keep only cryptocurrencies that are trading
csv_df = csv_df[csv_df['IsTrading'] == True]
csv_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [186]:
# Keep only cryptocurrencies with a working algorithm
csv_df = csv_df.loc[csv_df['Algorithm'].notnull()]
csv_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [187]:
# Remove the "IsTrading" column
csv_df = csv_df.drop(columns='IsTrading')
csv_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
SERO,Super Zero,Ethash,PoW,NaN,1000000000
UOS,UOS,SHA-256,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [188]:
# Remove rows with at least 1 null value
csv_df = csv_df.dropna()
csv_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [189]:
# Remove rows with cryptocurrencies having no coins mined
csv_df = csv_df[csv_df.TotalCoinSupply > '0']
csv_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [190]:
# Drop rows where there are 'N/A' text values
csv_df = csv_df[csv_df.iloc[:] != 'N/A'].dropna()
csv_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [191]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coin_name_df = pd.DataFrame(csv_df['CoinName'], index=csv_df.index)
coin_name_df

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
LTC,Litecoin
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex
ZEN,Horizen


In [192]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
csv_df = csv_df.drop(columns="CoinName")
csv_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
LTC,Scrypt,PoW,6.303924e+07,84000000
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Equihash,PoW,7.296538e+06,21000000


In [193]:
# Create dummy variables for text features
X = pd.get_dummies(data=csv_df, columns=["Algorithm", "ProofType"])
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LTC,6.303924e+07,84000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [194]:
# Standardize data
X = StandardScaler().fit_transform(X)

### Reducing Dimensions Using PCA

In [203]:
# Use PCA to reduce dimensions to 3 principal components
n_comp = 3
pca=PCA(n_comp)
pc_pca = pca.fit_transform(X)
pc_pca

array([[-0.10912567, -1.22917444, -0.34907782],
       [-0.09458888, -1.22656795, -0.34956078],
       [ 0.40182068, -1.89903168, -0.51525802],
       ...,
       [-0.2475705 ,  2.23816931,  0.21572143],
       [-0.3342742 ,  1.96349053,  0.22678391],
       [-0.01892285, -0.78942384, -0.1607484 ]])

In [204]:
# Create a DataFrame with the principal components data
col_names = [f"PC {i}" for i in range(1, n_comp + 1)]
pcs_df = pd.DataFrame(pc_pca, columns=col_names, index=csv_df.index)
print(pcs_df.shape)
pcs_df.head(10)

(634, 3)


,PC 1,PC 2,PC 3
42,-0.109126,-1.229174,-0.349078
404,-0.094589,-1.226568,-0.349561
1337,0.401821,-1.899032,-0.515258
BTC,-0.291273,1.398719,0.052050
LTC,-0.296971,1.042692,0.023177
DASH,-0.134666,-1.530323,-0.403952
ETC,-0.389840,1.979353,0.235920
ZEC,-0.334273,1.963491,0.226784
BTS,-0.055474,-2.018418,0.102651
DGB,-0.165823,1.735873,0.193486


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [205]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pc_pca)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {'K':k, "inertia":inertia}
elbow_df = pd.DataFrame(elbow_data)
elbow_df.hvplot.line(x="K", y= "inertia", title= "PCA Elbow Curve")

c:\ProgramData\Anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  f"KMeans is known to have a memory leak on Windows "


:Curve   [K]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [206]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)
# Fit the model
model.fit(pcs_df)
# Predict clusters
pred = model.predict(pcs_df)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = pd.concat([csv_df, pcs_df], axis=1, sort=False)
clustered_df["CoinName"] = coin_name_df["CoinName"]
clustered_df["Class"] = model.labels_
print(clustered_df.shape)
clustered_df.head(10)

(634, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.109126,-1.229174,-0.349078,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.094589,-1.226568,-0.349561,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,0.401821,-1.899032,-0.515258,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.291273,1.398719,0.052050,Bitcoin,1
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.296971,1.042692,0.023177,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.134666,-1.530323,-0.403952,Dash,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.389840,1.979353,0.235920,Ethereum Classic,1
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.334273,1.963491,0.226784,ZCash,1
BTS,SHA-512,PoS,2.741570e+09,3600570502,-0.055474,-2.018418,0.102651,Bitshares,0
DGB,Multiple,PoW,1.140622e+10,21000000000,-0.165823,1.735873,0.193486,DigiByte,1


### Visualizing Results

#### Scatter Plot with Tradable Cryptocurrencies

In [209]:
# Scale data to create the scatter plot
mm_scaler = MinMaxScaler()
plot_data = mm_scaler.fit_transform(
    clustered_df[["TotalCoinSupply", "TotalCoinsMined"]]
)
plot_df = pd.DataFrame(
    plot_data, columns=["TotalCoinSupply", "TotalCoinsMined"], index=clustered_df.index
)
plot_df["CoinName"] = clustered_df["CoinName"]
plot_df["Class"] = clustered_df["Class"]
plot_df.head()

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,0.000000e+00,0.005942,42 Coin,0
404,5.767955e-09,0.007002,404Coin,0
1337,3.406122e-06,0.035342,EliteCoin,0
BTC,2.276820e-10,0.005960,Bitcoin,1
LTC,9.107294e-10,0.006006,Litecoin,1


In [211]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
plot_df.hvplot.scatter(x="TotalCoinsMined",y="TotalCoinSupply", hover_cols= ["CoinName"], by = "Class")

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)

#### Table of Tradable Cryptocurrencies

In [213]:
# Table with tradable cryptos
clustered_df[
    [
        "CoinName",
        "Algorithm",
        "ProofType",
        "TotalCoinSupply",
        "TotalCoinsMined",
        "Class",
    ]
].hvplot.table()

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [214]:
# Print the total number of tradable cryptocurrencies
print(f'There are {clustered_df.shape[0]} tradable cryptocurrencies')

There are 634 tradable cryptocurrencies
